# Document Portal: Logging and Exception System Demo

This notebook demonstrates how to use the advanced logger and custom exception system implemented in the Document Portal.

In [1]:
import os
import sys

# Add the project root directory to sys.path to allow imports
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if project_root not in sys.path:
    sys.path.append(project_root)

print(f"Project root added to path: {project_root}")

Project root added to path: /home/aignishant/Documents/genaiproject/dp/document_portal


## 1. Advanced Logger

The logger outputs JSON formatted logs, which are easier to parse by log aggregation tools. It also supports adding contextual information.

In [2]:
from ai_common.logger.custom_logger import logger, get_logger
from ai_common.logger.logger_utils import add_context

# Basic logging
logger.info("This is a standard info log message.")
logger.warning("This is a warning message.")

INFO: This is a standard info log message.


In [3]:
# Adding context
# This creates a new logger adapter with the context bound to it
ctx_logger = add_context(logger, user_id="u-12345", request_id="req-abcde")

ctx_logger.info("User initiated a search")
ctx_logger.debug("Searching for documents", extra_params={"query": "machine learning"})

INFO: User initiated a search


## 2. Custom Exceptions

We have a hierarchy of exceptions inheriting from `AppException`. Each exception carries a standard error code and HTTP status code.

In [4]:
from ai_common.exception.custom_exception import (
    AppException,
    ResourceNotFoundException,
    ValidationException,
    AuthenticationException,
    PermissionDeniedException,
    DatabaseException
)

In [5]:
def demonstrate_exception(exception_class, message, **kwargs):
    try:
        print(f"Raising {exception_class.__name__}...")
        raise exception_class(message, details=kwargs)
    except AppException as e:
        print(f"Caught unexpected: {type(e).__name__}")
        print(f"  Status: {e.status_code}")
        print(f"  Code: {e.code}")
        print(f"  Message: {e.message}")
        print(f"  Details: {e.details}")
        print(f"  Serialized: {e.to_dict()}")
        print("-" * 40)

# Test ValidationException
demonstrate_exception(ValidationException, "Invalid email format", field="email", value="bob@domain")

# Test ResourceNotFoundException
demonstrate_exception(ResourceNotFoundException, "Document not found", doc_id=999)

Raising ValidationException...
Caught unexpected: ValidationException
  Status: 400
  Code: VALIDATION_ERROR
  Message: Invalid email format
  Details: {'field': 'email', 'value': 'bob@domain'}
  Serialized: {'error': {'code': 'VALIDATION_ERROR', 'message': 'Invalid email format', 'details': {'field': 'email', 'value': 'bob@domain'}}}
----------------------------------------
Raising ResourceNotFoundException...
Caught unexpected: ResourceNotFoundException
  Status: 404
  Code: RESOURCE_NOT_FOUND
  Message: Document not found
  Details: {'doc_id': 999}
  Serialized: {'error': {'code': 'RESOURCE_NOT_FOUND', 'message': 'Document not found', 'details': {'doc_id': 999}}}
----------------------------------------


## 3. Integration: Logging Exceptions

The generic `AppException` has a helper method `log_error()` that logs the exception details using the standard logger.

In [6]:
try:
    # Simulate a critical failure
    raise DatabaseException("Connection pool exhausted")
except DatabaseException as e:
    # Log the error with all its structured data
    e.log_error()
    print("Exception logged. Check the console/log file to see the structured JSON output.")

ERROR: Connection pool exhausted


Exception logged. Check the console/log file to see the structured JSON output.
